In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Checking Partial Derivatives with Finite Difference
In addition to using approximations to estimate partial derivatives, you can also use approximations to check your implementations of the partial derivatives for a component.

`Problem` has a method, `check_partials`, that checks partial derivatives
comprehensively for all Components in your model. To do this check, the framework compares the analytic result against a finite difference result. This means that the check_partials function can be quite computationally expensive. So use it to check your work, but don’t leave the call in your production run scripts.

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.check_partials
        :noindex:
```

```{Note}
For components that provide their partials directly (from the `compute_partials` or `linearize` methods, only information about the forward derivatives are shown. For components that are matrix-free, both forward and reverse derivative information is shown.

Implicit components are matrix-free if they define a `apply_linear` method. Explicit components are matrix-free if they define a `compute_jacvec_product` method.
``` 

## Basic Usage
When the difference between the FD derivative and the provided derivative is larger (in either a relative or absolute sense) than 1e-6, that partial derivative will be marked with a '*'.

In [2]:
import numpy as np

import openmdao.api as om

class MyComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)

        self.add_output('y', 5.5)

        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x1'] + 4.0*inputs['x2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['y', 'x1'] = np.array([4.0])
        J['y', 'x2'] = np.array([40])

prob = om.Problem()

prob.model.add_subsystem('comp', MyComp())

prob.set_solver_print(level=0)

prob.setup(mode='rev')
prob.run_model()

data = prob.check_partials()

------------------------
Component: MyComp 'comp'
------------------------

  comp: 'y' wrt 'x1'
     Forward Magnitude: 4.000000e+00
          Fd Magnitude: 3.000000e+00 (fd:forward)

    Absolute Error (Jfor - Jfd) : 1.000000e+00 *

    Relative Error (Jfor - Jfd) / Jfd : 3.333333e-01 *

    Raw Forward Derivative (Jfor)
    [[4.]]

    Raw FD Derivative (Jfd)
    [[3.]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  comp: 'y' wrt 'x2'
     Forward Magnitude: 4.000000e+01
          Fd Magnitude: 4.000000e+00 (fd:forward)

    Absolute Error (Jfor - Jfd) : 3.600000e+01 *

    Relative Error (Jfor - Jfd) / Jfd : 9.000000e+00 *

    Raw Forward Derivative (Jfor)
    [[40.]]

    Raw FD Derivative (Jfd)
    [[4.]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


In [3]:
x1_error = data['comp']['y', 'x1']['abs error']

print(x1_error.forward)

1.0000000004688445


In [4]:
x2_error = data['comp']['y', 'x2']['rel error']

print(x2_error.forward)

8.99999999860222


In [5]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(x1_error.forward, 1.0000000004688445)
assert_near_equal(x2_error.forward, 8.99999999860222)

0.0

Turn off standard output and just view the derivatives in the return:

In [6]:
import numpy as np

import openmdao.api as om

class MyComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)

        self.add_output('y', 5.5)

        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x1'] + 4.0*inputs['x2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['y', 'x1'] = np.array([4.0])
        J['y', 'x2'] = np.array([40])

prob = om.Problem()

prob.model.add_subsystem('comp', MyComp())

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

In [7]:
data = prob.check_partials(out_stream=None, compact_print=True)
print(data)

{'comp': {('y', 'x1'): {'J_fwd': array([[4.]]), 'J_fd': array([[3.]]), 'abs error': ErrorTuple(forward=1.0000000004688445, reverse=None, forward_reverse=None), 'rel error': ErrorTuple(forward=0.3333333335417087, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=4.0, reverse=None, fd=2.9999999995311555)}, ('y', 'x2'): {'J_fwd': array([[40.]]), 'J_fd': array([[4.]]), 'abs error': ErrorTuple(forward=35.99999999944089, reverse=None, forward_reverse=None), 'rel error': ErrorTuple(forward=8.99999999860222, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=40.0, reverse=None, fd=4.000000000559112)}}}


# Show Only Incorrect Printing Option
If you are only concerned with seeing the partials calculations that are incorrect, set `show_only_incorrect` to `True`. This applies to both `compact_print` equal to `True` and `False`.

In [8]:
import numpy as np
import openmdao.api as om

class MyCompGoodPartials(om.ExplicitComponent):
    def setup(self):
        self.add_input('x1', 3.0)
        self.add_input('x2', 5.0)
        self.add_output('y', 5.5)
        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0 * inputs['x1'] + 4.0 * inputs['x2']

    def compute_partials(self, inputs, partials):
        """Correct derivative."""
        J = partials
        J['y', 'x1'] = np.array([3.0])
        J['y', 'x2'] = np.array([4.0])

class MyCompBadPartials(om.ExplicitComponent):
    def setup(self):
        self.add_input('y1', 3.0)
        self.add_input('y2', 5.0)
        self.add_output('z', 5.5)
        self.declare_partials(of='*', wrt='*')

    def compute(self, inputs, outputs):
        outputs['z'] = 3.0 * inputs['y1'] + 4.0 * inputs['y2']

    def compute_partials(self, inputs, partials):
        """Intentionally incorrect derivative."""
        J = partials
        J['z', 'y1'] = np.array([33.0])
        J['z', 'y2'] = np.array([40.0])

prob = om.Problem()
prob.model.add_subsystem('good', MyCompGoodPartials())
prob.model.add_subsystem('bad', MyCompBadPartials())
prob.model.connect('good.y', 'bad.y1')

prob.set_solver_print(level=0)
prob.setup()
prob.run_model()

prob.check_partials(compact_print=True, show_only_incorrect=True)
prob.check_partials(compact_print=False, show_only_incorrect=True)


** Only writing information about components with incorrect Jacobians **

----------------------------------
Component: MyCompBadPartials 'bad'
----------------------------------

+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+
| of '<variable>' | wrt '<variable>' |   calc mag. |  check mag. |  a(cal-chk) |  r(cal-chk) | error desc         |
+=================+==================+=============+=============+=============+=============+====================+
| 'z'             | 'y1'             |  3.3000e+01 |  3.0000e+00 |  3.0000e+01 |  1.0000e+01 |  >ABS_TOL >REL_TOL |
+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+
| 'z'             | 'y2'             |  4.0000e+01 |  4.0000e+00 |  3.6000e+01 |  9.0000e+00 |  >ABS_TOL >REL_TOL |
+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+

######

{'good': {('y', 'x1'): {'J_fwd': array([[3.]]),
   'J_fd': array([[3.]]),
   'abs error': ErrorTuple(forward=4.688445187639445e-10, reverse=None, forward_reverse=None),
   'rel error': ErrorTuple(forward=1.5628150627907208e-10, reverse=None, forward_reverse=None),
   'magnitude': MagnitudeTuple(forward=3.0, reverse=None, fd=2.9999999995311555)},
  ('y', 'x2'): {'J_fwd': array([[4.]]),
   'J_fd': array([[4.]]),
   'abs error': ErrorTuple(forward=5.591118679149076e-10, reverse=None, forward_reverse=None),
   'rel error': ErrorTuple(forward=1.3977796695918903e-10, reverse=None, forward_reverse=None),
   'magnitude': MagnitudeTuple(forward=4.0, reverse=None, fd=4.000000000559112)}},
 'bad': {('z', 'y1'): {'J_fwd': array([[33.]]),
   'J_fd': array([[3.00000001]]),
   'abs error': ErrorTuple(forward=29.999999993363417, reverse=None, forward_reverse=None),
   'rel error': ErrorTuple(forward=9.999999975665864, reverse=None, forward_reverse=None),
   'magnitude': MagnitudeTuple(forward=33.0, re

# Running With Multiple FD Step Sizes
If the `step` argument is provided as a list of values instead of a single value, the FD partial derivatives will be evaluated and displayed for each given step size.  This can be useful in certain cases where complex step checks are not possible and the component(s) being checked are expensive to execute.  Supplying multiple FD step sizes in that case will only compute the analytic derivatives once and compare those to each computed FD derivative, which is less expensive than making a separate call to `check_partials` for each FD step size.